Author: Mark Searle  
Contact: bg223@stud.uni-heidelberg.de

# PyHelios Demo: LiDAR simulation trajectory plot.

***

This notebook demos the excecution of a __LiDAR survey simulation__ using integrated Python functionalities to initiate __Helios++__.
The trajectory of the virtual scanner which can be configured to sit upon different moving or stationary platforms is then plotted using __matplotlib__.

***
The survey is configured through a __.xml survey file__ containing survey name, platform type, scanner type, and scene to be scanned along with platform and scanner settings.  
The simulation runs in several legs which represent individual scanner trajectories and are also configured from within survey file.

The survey files are located within __'helios/data/surveys'__!

The __survey used in this demo__ (`custom_als_toyblocks.xml`) should be located in the same folder as this notebook.

***

## Python path extension:

In order for your python installation to recognize pyhelios, the '.../run' path must be added to your __python path__.
You can do this each time manually with `sys.path.append(runpath)` or you can __permanently add the run path__ to your python installation.

To do so on __Windows__, go to your Python setup folder and open Lib/site-packages. Add an __'example.pth'__ file to this directory and add the required path to the file.

***

<img src="images/example_path_content.JPG" width="400" style="float:left"/> 
<img src="images/example_path_file.JPG" width="400" />  

***

`# Example temporary solution:
example_runpath = 'helios/run'
import sys
sys.path.append(example_runpath)`
    
***

__Test by importing pyhelios:__

In [2]:
# Imports
import time
import os
import sys

In [3]:
# Change working directory to helios path and configure runpath within pyhelios.
helios_path = 'your_path/202007_helios_bin/'
helios_run_path = 'run/'
# Survey to be used. (path from within helios)
survey_path = 'toyblocks/custom_als_toyblocks.xml'

# Change working directory.
os.chdir(helios_path)

# Add run path to python path.
sys.path.append(helios_run_path)

In [4]:
# PyHelios import
import pyhelios

__NOTE:__ Changing working directory to helios path only necessary when running notebook from outside helios path. Run path must be added before importing pyhelios, unless permanent .pth solution applied.

## LiDAR Simulation with PyHelios
__1st step - Initiate and run simulation. Access output using pyhelios functions:__

***

1. Configure simulation context and build a simulation:

In [5]:
# Sim context.
# Set logging.
pyhelios.loggingVerbose2()
# Set seed for random number generator.
pyhelios.setDefaultRandomnessGeneratorSeed("123")

In [6]:
# Build simulation parameters: (surveyPath, assetsPath, outputPath, ...).
sim = pyhelios.Simulation(
    'data/surveys/' + survey_path,
    'assets/',
    'output/',
    0,          # Num Threads
    False,      # LAS output
    False,      # ZIP output
    )

# Enable final output.
sim.finalOutput = True

2. Load survey file:

In [61]:
# Set sim frequency
# Has to be set, default value is 0. With a frequency of 0, pausing and callbacks are not possible.
sim.simFrequency = 100
print('SimFrequency : {sf}'.format(sf=sim.simFrequency))

# Load survey file. Further configuration of survey possible.
sim.loadSurvey(
        True,       # Leg Noise Disabled FLAG
        False,      # Rebuild Scene FLAG
        False,       # Write Waveform FLAG
        True,       # Calc Echowidth FLAG
        False,      # Full Wave Noise FLAG
        True        # Platform Noise Disabled FLAG
            )

SimFrequency : 100


3. Run PyHelios simulation:

In [62]:
# Start the simulation.
sim.start()

# Various simulation status functions available.
if sim.isStarted():
    print('Simulation has started!\nSurvey Name: {survey_name}\n{scanner_info}'.format(
        survey_name = sim.getSurvey().name,
        scanner_info = sim.getScanner().toString()))

# Simulation can be paused with simulation.pause().
time.sleep(10)
sim.pause()

if sim.isPaused():
    print('Simulation is paused!')
    
if not sim.isRunning():
    print('Sim is not running.')
    
time.sleep(5)
sim.resume()

if sim.isRunning():
    print('Simulation has resumed!')

Simulation has started!
Survey Name: custom_als_toyblocks
Scanner: riegl_vq-880g Power: 4.000000 W Divergence: 0.300000 mrad Wavelength: 1064 nm Visibility: 23.000000 km
Simulation is paused!
Sim is not running.
Simulation has resumed!


4. Process Output:

In [64]:
# Create instance of PyHeliosOutputWrapper class using sim.join(). 
# Contains attributes 'measurements' and 'trajectories' which are Python wrappers of classes that contain the output vectors.
output = sim.join()

# Create instances of vector classes by accessing 'measurements' and 'trajectories' attributes of output wrapper.
measurements = output.measurements
trajectories = output.trajectories

# Get amount of points in trajectory and amount of measurements by accessing length of measurement and trajectory vectors.
print('Number of measurements : {n}'.format(n=len(measurements)))
print('Number of points in trajectory : {n}'.format(n=len(trajectories)))

# Each element of vectors contains a measurement point or point in trajectory respectively. Access through getPosition().
# Get starting and end point of trajectory from first and last element of trajectory with getPosition() method.
starting_point = trajectories[0].getPosition()
end_point = trajectories[len(trajectories) - 1].getPosition()

# Output individual x, y and z vals.
# Accessed through x, y and z attributes of points from getPosition() method.
print('Trajectory starting point : ({x}, {y}, {z})'.format(
    x=starting_point.x, y=starting_point.y, z=starting_point.z))

print('Trajectory end point : ({x}, {y}, {z})'.format(
    x=end_point.x, y=end_point.y, z=end_point.z))

Number of measurements : 2393455
Number of points in trajectory : 1867
Trajectory starting point : (-50.0, -50.0, 80.0)
Trajectory end point : (49.58522486062412, 49.9987565158913, 80.0)


## Plot Resulting Trajectory
__2nd step: Survey scanner trajectory plot using matplotlib mplot3d:__

***

Here we can make use of __pyheliostools__, a library of functions associated with pyhelios. It should be included in your helios package. Specifically, we will use the function __outputToNumpy__, which saves both measurement and trajectory data as numpy Arrays.  

In [65]:
# Imports
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# PyHeliostools
from pyheliostools import outputToNumpy

1. Create numpy Array with points from trajectory using `pyheliostools.outputToNumpy()`:

In [66]:
# Output is two np.arrays. One for measurements and one for trajectory points.
measurement_points, trajectory_points = outputToNumpy(output)

# Points to be plotted:
# First three cols are x, y and z vals
points=trajectory_points[:,0:3]

4. Groundplane array for the sake of more comprehensive visualization:

In [67]:
# Empty list for groundplane vals.
groundplane_list = []

# List filled every 25th value from (-75, -75, 0) to (75, 75, 0).
for i in range(-75, 76, 25):
    for j in range(-75, 76, 25):
        groundplane_list.append([i, j, 0])
        
# List converted to numpy Array.
groundplane = np.array(groundplane_list)

4. 3D figure is created using mplot3d and first three cols of 'points' array (format is x, y, z) are plotted onto 3D figure:

In [50]:
# Magic command to enable interactive plot.
%matplotlib notebook

# Matplotlib figure.
fig = plt.figure()
# Axes3d axis onto mpl figure.
ax = fig.add_subplot(projection='3d')

# Plot scatterplot of trajectory.
ax.plot(points[:,0], points[:,1], points[:,2], c = 'green', label = 'scanner trajectory')
# Surface plot of groundplane.
ax.plot_trisurf(groundplane[:,0], groundplane[:,1], groundplane[:,2], color='darkgoldenrod', label='groundplane')

# Add fancy labels to scatterplots (alternative to legend).
ax.text(points[-1,0], points[-1,1], points[-1,2], 'trajectories',
        bbox = dict(boxstyle='round', fc="w", ec="k"), size = '7')
ax.text(groundplane[-1,0], groundplane[-1,1], groundplane[-1,2], 'groundplane',
        bbox = dict(boxstyle='round', fc="w", ec="k"), size = '7')

# Add axis labels.
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.set_zlabel('$Z$')

# Set title.
ax.set_title(label='LiDAR Scanner Trajectory from PyHelios Simulation')
# Set subtitle.
ax.text2D(0.1, 0.97, "survey: {s}, points in trajectory: {n}".format(s=sim.getSurvey().name, n=len(trajectories)),
                                                                              fontsize='8', transform=ax.transAxes)
# Display results
plt.show()

<IPython.core.display.Javascript object>